![Photo by Stephen Phillips - Hostreviews.co.uk on UnSplash](https://cf.bstatic.com/xdata/images/hotel/max1024x768/408003083.jpg?k=c49b5c4a2346b3ab002b9d1b22dbfb596cee523b53abef2550d0c92d0faf2d8b&o=&hp=1){fig-align="center" width=50%}


# Import data

In [1]:
import time
import warnings
from pathlib import Path

import catboost
import lightgbm as lgb
import numpy as np
import pandas as pd
import xgboost
from data import utils
from IPython.display import clear_output, display
from lets_plot import *
from lets_plot.mapping import as_discrete
from sklearn import (
    compose,
    dummy,
    ensemble,
    impute,
    linear_model,
    model_selection,
    pipeline,
    preprocessing,
    svm,
    tree,
)
from sklearn.base import BaseEstimator, TransformerMixin
from tqdm import tqdm

LetsPlot.setup_html()

**Objective**:
* Examine the necessary sample pre-processing steps before modeling
* Create the required pipeline
* 
Evaluate multiple algoritms
* 
Choose a suitable baseli modell.





# Prepare dataframe before modelling
## Read in the processed file

After importing our preprocessed dataframe, a crucial step in our data refinement process involves the culling of certain columns. Specifically, we intend to exclude columns with labels such as "external_reference," "ad_url," "day_of_retrieval," "website," "reference_number_of_the_epc_report," and "housenumber." Our rationale behind this action is to enhance the efficiency of our model by eliminating potentially non-contributory features.

In [2]:
utils.seed_everything(utils.Configuration.seed)

df = (
    pd.read_parquet(
        utils.Configuration.INTERIM_DATA_PATH.joinpath(
            "2023-10-01_Processed_dataset_for_NB_use.parquet.gzip"
        )
    )
    .sample(frac=1, random_state=utils.Configuration.seed)
    .reset_index(drop=True)
    .assign(price=lambda df: np.log(df.price))
    .drop(
        columns=[
            "external_reference",
            "ad_url",
            "day_of_retrieval",
            "website",
            "reference_number_of_the_epc_report",
            "housenumber",
        ]
    )
)

print(f"Shape of dataframe after read-in a pre-processing: {df.shape}")
X = df.drop(columns=utils.Configuration.target_col)
y = df[utils.Configuration.target_col]

print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}")

Shape of dataframe after read-in a pre-processing: (3660, 50)
Shape of X: (3660, 49)
Shape of y: (3660,)


## Train-test split

The subsequent phase in our data preparation involves the partitioning of our dataset into training and testing subsets. To accomplish this, we'll leverage the `model_selection.train_test_split` method. This step ensures that we have distinct sets for model training and evaluation, a fundamental practice in machine learning.

In [3]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X, y, test_size=0.2, random_state=utils.Configuration.seed
)

print(f"Shape of X-train: {X_train.shape}")
print(f"Shape of X-test: {X_test.shape}")

Shape of X-train: (2928, 49)
Shape of X-test: (732, 49)


# Implementing the data-processing pipeline

In order to compare various machine learning algorithms effectively, our initial approach will involve constructing a straightforward pipeline. This pipeline's primary objective is to segregate columns based on their data types, recognizing the need for distinct preprocessing steps for continuous (numerical) and categorical variables. To facilitate this process within our scikit-learn pipeline, we will begin by implementing a custom class named "FeatureSelector."

The rationale behind thor is to establish a structured approach to feature handlils. The FeatureSelector class will provide us with a streamlined means to access and process columns based on their data typess.

In [4]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    """
    A transformer for selecting specific columns from a DataFrame.

    This class inherits from the BaseEstimator and TransformerMixin classes from sklearn.base.
    It overrides the fit and transform methods from the parent classes.

    Attributes:
        feature_names_in_ (list): The names of the features to select.
        n_features_in_ (int): The number of features to select.

    Methods:
        fit(X, y=None): Fit the transformer. Returns self.
        transform(X, y=None): Apply the transformation. Returns a DataFrame with selected features.
    """

    def __init__(self, feature_names_in_):
        """
        Constructs all the necessary attributes for the FeatureSelector object.

        Args:
            feature_names_in_ (list): The names of the features to select.
        """
        self.feature_names_in_ = feature_names_in_
        self.n_features_in_ = len(feature_names_in_)

    def fit(self, X, y=None):
        """
        Fit the transformer. This method doesn't do anything as no fitting is necessary.

        Args:
            X (DataFrame): The input data.
            y (array-like, optional): The target variable. Defaults to None.

        Returns:
            self: The instance itself.
        """
        return self

    def transform(self, X, y=None):
        """
        Apply the transformation. Selects the features from the input data.

        Args:
            X (DataFrame): The input data.
            y (array-like, optional): The target variable. Defaults to None.

        Returns:
            DataFrame: A DataFrame with only the selected features.
        """
        return X.loc[:, self.feature_names_in_].copy(deep=True)

In [5]:
# Selecting columns by dtypes

numerical_columns = X_train.head().select_dtypes("number").columns.to_list()
categorical_columns = X_train.head().select_dtypes("object").columns.to_list()

Addressing missing values is a crucial preliminary step in our machine learning pipeline, as certain algorithms are sensitive to data gaps. To handle this, we'll employ imputation techniques tailored to the data types of the columns.

For numerical columns, we'll adopt the "median" strategy for imputation. This approach involves replacing missing values with the median of the available data in the respective numerical column. It's a robust choice for handling missing values in numerical data as it's less sensitive to outliers.

Conversely, for categorical columns, we'll opt for imputation using the most frequent values in each column. By filling in missing categorical data with the mode (most common value) for that column, we ensure that the imputed values align with the existing categorical distribution, preserving the integrity of the categorical features.

This systematic approach to imputation sets a solid foundation for subsequent machine learning algorithms, ensuring that our dataset is well-prepared for analysis and modeling.

In [6]:
# Prepare pipelines for corresponding columns:
numerical_pipeline = pipeline.Pipeline(
    steps=[
        ("num_selector", FeatureSelector(numerical_columns)),
        ("imputer", impute.SimpleImputer(strategy="median")),
        ("std_scaler", preprocessing.MinMaxScaler()),
    ]
)

categorical_pipeline = pipeline.Pipeline(
    steps=[
        ("cat_selector", FeatureSelector(categorical_columns)),
        ("imputer", impute.SimpleImputer(strategy="most_frequent")),
        (
            "onehot",
            preprocessing.OneHotEncoder(handle_unknown="ignore", sparse_output=True),
        ),
    ]
)

Once we are satisfied with the individual pipelines designed for numerical and categorical feature processing, the next step involves merging them into a unified pipeline using the `FeatureUnion` method provided by scikit-learn.

In [7]:
# Put all the pipelines inside a FeatureUnion:
data_preprocessing_pipeline = pipeline.FeatureUnion(
    n_jobs=-1,
    transformer_list=[
        ("numerical_pipeline", numerical_pipeline),
        ("categorical_pipeline", categorical_pipeline),
    ],
)

# Compare the performance of several algorithms

Bringing all these components together in our machine learning pipeline is the culmination of our data preparation and model evaluation process. 

1. **Algorithm Selection**We c Choose a set of machine learning algorithms thaweou want to evaluatek.

2. **Data SplittingWe u*: Utilize the `ShuffleSplit` method to generate randomized indices for splittouryour data into training and test sets. This ensures randomness in data selection and is crucial for unbiased evaluation.

3. **Model Training and Evaluation**: For each selected algorwe followfollow these steps:
   - Fit the model on the training data.
   - Evaluate the model using negative mean squared error (`neg_mean_squared_error`) as the scoring metric.
   - Record the training and test scores, as well as the standard deviation of scores to assess model s
   - Measure the time taken to fit each model, which provides insights into computational efficiency.el performance.

In [8]:
with warnings.catch_warnings():
    warnings.simplefilter(action="ignore", category=FutureWarning)

    MLA = [
        linear_model.LinearRegression(),
        linear_model.SGDRegressor(),
        linear_model.PassiveAggressiveRegressor(),
        linear_model.RANSACRegressor(),
        linear_model.Lasso(),
        svm.SVR(),
        ensemble.GradientBoostingRegressor(),
        tree.DecisionTreeRegressor(),
        ensemble.RandomForestRegressor(),
        ensemble.ExtraTreesRegressor(),
        ensemble.AdaBoostRegressor(),
        catboost.CatBoostRegressor(silent=True),
        lgb.LGBMRegressor(verbose=-1),
        xgboost.XGBRegressor(verbosity=0),
        dummy.DummyClassifier(),
    ]

    # note: this is an alternative to train_test_split
    cv_split = model_selection.ShuffleSplit(
        n_splits=10, test_size=0.3, train_size=0.6, random_state=0
    )  # run model 10x with 60/30 split intentionally leaving out 10%

    # create table to compare MLA metrics
    MLA_columns = [
        "MLA Name",
        "MLA Parameters",
        "MLA Train RMSE Mean",
        "MLA Test RMSE Mean",
        "MLA Test RMSE 3*STD",
        "MLA Time",
    ]
    MLA_compare = pd.DataFrame(columns=MLA_columns)

    # index through MLA and save performance to table
    row_index = 0
    for alg in tqdm(MLA):
        # set name and parameters
        MLA_name = alg.__class__.__name__
        MLA_compare.loc[row_index, "MLA Name"] = MLA_name
        MLA_compare.loc[row_index, "MLA Parameters"] = str(alg.get_params())

        model_pipeline = pipeline.Pipeline(
            steps=[
                ("data_preprocessing_pipeline", data_preprocessing_pipeline),
                ("model", alg),
            ]
        )

        cv_results = model_selection.cross_validate(
            model_pipeline,
            X_train,
            y_train,
            cv=cv_split,
            scoring="neg_mean_squared_error",
            return_train_score=True,
        )

        MLA_compare.loc[row_index, "MLA Time"] = cv_results["fit_time"].mean()
        MLA_compare.loc[row_index, "MLA Train RMSE Mean"] = cv_results[
            "train_score"
        ].mean()
        MLA_compare.loc[row_index, "MLA Test RMSE Mean"] = cv_results[
            "test_score"
        ].mean()
        MLA_compare.loc[row_index, "MLA Test RMSE 3*STD"] = (
            cv_results["test_score"].std() * 3
        )

        row_index += 1

        clear_output(wait=True)
        display(MLA_compare.sort_values(by=["MLA Test RMSE Mean"], ascending=False))

,MLA Name,MLA Parameters,MLA Train RMSE Mean,MLA Test RMSE Mean,MLA Test RMSE 3*STD,MLA Time
11,CatBoostRegressor,"{'loss_function': 'RMSE', 'silent': True}",-0.016777,-0.069782,0.0099,3.244442
12,LGBMRegressor,"{'boosting_type': 'gbdt', 'class_weight': None...",-0.012073,-0.077218,0.006721,0.1577
9,ExtraTreesRegressor,"{'bootstrap': False, 'ccp_alpha': 0.0, 'criter...",-0.000005,-0.085609,0.011404,21.911669
6,GradientBoostingRegressor,"{'alpha': 0.9, 'ccp_alpha': 0.0, 'criterion': ...",-0.050419,-0.085817,0.012371,1.716229
13,XGBRegressor,"{'objective': 'reg:squarederror', 'base_score'...",-0.005383,-0.087365,0.013097,0.207547
8,RandomForestRegressor,"{'bootstrap': True, 'ccp_alpha': 0.0, 'criteri...",-0.012561,-0.089146,0.012937,18.576334
10,AdaBoostRegressor,"{'base_estimator': 'deprecated', 'estimator': ...",-0.104764,-0.12835,0.014058,0.738398
5,SVR,"{'C': 1.0, 'cache_size': 200, 'coef0': 0.0, 'd...",-0.051323,-0.129455,0.023128,0.393076
7,DecisionTreeRegressor,"{'ccp_alpha': 0.0, 'criterion': 'squared_error...",-0.000005,-0.1877,0.035284,0.325504
2,PassiveAggressiveRegressor,"{'C': 1.0, 'average': False, 'early_stopping':...",-0.110837,-0.380512,0.822766,0.044286


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [11:25<00:00, 45.68s/it]


It's evident from the table above that the `CatBoostRegressor` has demonstrated the most promising performance, achieving a score of 0.0697 on the test set. Following closely are the `LGBMRegressor`, `ExtraTreesRegressor`, and `GradientBoostingRegressor`, with `XGBRegressor` trailing behind.

In the next section, we'll delve deeper into the optimization of our model. This exploration will include refining model settings, crafting improved features, and employing various techniques to enhance overall predictive accuracy. See you in the next section!